In [1]:
import importlib as imp
import sys
import numpy as np
sys.path = list(set(sys.path + ['../../']))

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"
import tensorflow as tf

In [3]:
from sklearn import datasets
dataset_iris = datasets.load_iris(return_X_y=True)

In [4]:
def generate_sample_dataset_and_feature_dict(dataset_iris):
    """
        Make dataset from iris, assuming a user likes class 0 but dislikes class 1 and class 2
    """
    npd = np.zeros(len(dataset_iris[0]), dtype=[
        ('user_id', np.int32), ('item_id', np.int32), ('label', np.int32)])
    fea_dict = dict()
    for idx in range(len(dataset_iris[0])):
        if dataset_iris[1][idx] == 0:
            npd[idx] = (0, idx, 1)
        else:
            npd[idx] = (0, idx, -1)
        fea_dict[idx] = dataset_iris[0][idx]
    return npd, fea_dict

data_all, fea_dict = generate_sample_dataset_and_feature_dict(dataset_iris)

In [5]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data_all)
print(len(data_train), len(data_test))

112 38


In [6]:
# create dataset instance from structured ndarray data
import recsys.dataset as dataset
dataset = imp.reload(dataset)
train_dataset = dataset.Dataset(data_train, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Train')
test_dataset = dataset.Dataset(data_test, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Test')

In [7]:
# create featurizer for mapping: (user, item) -> vector
import recsys.featurizer as featurizer
featurizer = imp.reload(featurizer)
class Featurizer(featurizer.FeaturizerBase):
    """
        Custom featurizer
    """
    def __init__(self, fea_dict):
        self.fea_dict = fea_dict

    def feature_dim(self):
        """
            Dim of X
        """
        return 4
    
    def featurize(self, user_id, item_id):
        if user_id != 0:
            raise "error"
        return self.fea_dict[item_id]

featurizer_iris = Featurizer(fea_dict)

In [8]:
# create data sampler for training and testing
import recsys.samplers.ranknet_sampler as ranknet_sampler
ranknet_sampler = imp.reload(ranknet_sampler)
train_sampler = ranknet_sampler.create_training_sampler(
    dataset=train_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, batch_size=10, num_process=1, seed=100)
test_sampler = ranknet_sampler.create_evaluation_sampler(
    dataset=test_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, seed=10)

In [9]:
# create evaluation metric
import recsys.metrics.auc as auc
auc = imp.reload(auc)
auc_evaluator = auc.AUC()

In [10]:
# create model
import recsys.recommenders.factorization_machine_recommender as factorization_machine_recommender
factorization_machine_recommender = imp.reload(factorization_machine_recommender)
model = factorization_machine_recommender.FactorizationMachineRecommender(feature_dim=featurizer_iris.feature_dim(),
    factor_dim=5, init_model_dir=None, save_model_dir='FMRec', l2_reg=None, train=True, serve=True)

In [11]:
# create model trainer with model
import recsys.model_trainer as model_trainer
model_trainer = imp.reload(model_trainer)
trainer = model_trainer.ModelTrainer(model=model)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
import datetime
print(datetime.datetime.now())

2019-08-02 10:37:21.716955


In [13]:
# train
trainer.train(total_iter=100, 
                    eval_iter=1,
                    save_iter=5,
                    train_sampler=train_sampler,
                    eval_samplers=[test_sampler], 
                    evaluators=[auc_evaluator])

[Training starts, total_iter: 100, eval_iter: 1, save_iter: 5]
[iter 1] loss: 1.694271
..(dataset: Test) evaluation
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 2] loss: 1.441551
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 3] loss: 1.651567
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 4] loss: 1.426173
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 5] Model saved.
[iter 5] loss: 1.457178
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 6] loss: 1.571098
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0


INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 47] loss: 0.989375
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 48] loss: 1.034003
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.039999999999999994
[iter 49] loss: 0.903284
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
Instructions for updating:
Use standard file APIs to delete files with this prefix.
[iter 50] Model saved.
[iter 50] loss: 0.938672
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 51] loss: 1.066978
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.0
[iter 52] loss: 0.799517
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from 

INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.96
[iter 92] loss: 0.316228
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 93] loss: 0.252094
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 94] loss: 0.156973
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 95] Model saved.
[iter 95] loss: 0.111624
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 96] loss: 0.179324
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 97] loss: 0.169706
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 0.96
[iter 98] loss: 0.141639
..(dataset: Test) evaluation
IN

In [14]:
import datetime
print(datetime.datetime.now())

2019-08-02 10:37:36.676371


In [15]:
# Export Protobuf model for online serving
!rm -rf ./pbModel
model.export(export_model_dir="pbModel", as_text=False)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: pbModel/saved_model.pb


In [16]:
# inspect internal op tensor value of training
b = train_sampler.next_batch()
print("data:", b)
print("dy, dy_tilde:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.get_outputs()))
print("loss:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.get_losses()))

data: [([5. , 3.5, 1.6, 0.6], [6.4, 2.7, 5.3, 1.9], 1)
 ([6.4, 2.7, 5.3, 1.9], [5. , 3.5, 1.6, 0.6], 0)
 ([5. , 3.5, 1.6, 0.6], [6.5, 2.8, 4.6, 1.5], 1)
 ([6.5, 2.8, 4.6, 1.5], [5. , 3.5, 1.6, 0.6], 0)
 ([5. , 3.5, 1.6, 0.6], [5.5, 2.6, 4.4, 1.2], 1)
 ([5.5, 2.6, 4.4, 1.2], [5. , 3.5, 1.6, 0.6], 0)
 ([5. , 3.5, 1.6, 0.6], [6. , 2.2, 5. , 1.5], 1)
 ([6. , 2.2, 5. , 1.5], [5. , 3.5, 1.6, 0.6], 0)
 ([5. , 3.5, 1.6, 0.6], [6.9, 3.2, 5.7, 2.3], 1)
 ([6.9, 3.2, 5.7, 2.3], [5. , 3.5, 1.6, 0.6], 0)
 ([4.9, 3. , 1.4, 0.2], [6.4, 2.7, 5.3, 1.9], 1)
 ([6.4, 2.7, 5.3, 1.9], [4.9, 3. , 1.4, 0.2], 0)
 ([4.9, 3. , 1.4, 0.2], [6.5, 2.8, 4.6, 1.5], 1)
 ([6.5, 2.8, 4.6, 1.5], [4.9, 3. , 1.4, 0.2], 0)
 ([4.9, 3. , 1.4, 0.2], [5.5, 2.6, 4.4, 1.2], 1)
 ([5.5, 2.6, 4.4, 1.2], [4.9, 3. , 1.4, 0.2], 0)
 ([4.9, 3. , 1.4, 0.2], [6. , 2.2, 5. , 1.5], 1)
 ([6. , 2.2, 5. , 1.5], [4.9, 3. , 1.4, 0.2], 0)
 ([4.9, 3. , 1.4, 0.2], [6.9, 3.2, 5.7, 2.3], 1)
 ([6.9, 3.2, 5.7, 2.3], [4.9, 3. , 1.4, 0.2], 0)]
dy, dy_tilde:

In [17]:
# inspect internal op tensor value of testing
b = test_sampler.next_batch()
print("data:", b)
print("score:", model.serve_inspect_ports(batch_data=b[1], ports=model.servegraph.get_outputs()))

data: ([1, 1, 1, 1, 1, -1, -1, -1, -1, -1], array([([4.3, 3. , 1.1, 0.1],), ([4.4, 3.2, 1.3, 0.2],),
       ([4.7, 3.2, 1.3, 0.2],), ([4.7, 3.2, 1.6, 0.2],),
       ([4.8, 3.4, 1.9, 0.2],), ([6.7, 3.1, 4.7, 1.5],),
       ([6.3, 2.9, 5.6, 1.8],), ([6.9, 3.1, 4.9, 1.5],),
       ([6.5, 3. , 5.5, 1.8],), ([4.9, 2.4, 3.3, 1. ],)],
      dtype=[('x', '<f4', (4,))]))
score: [array([-1.5017946, -1.6809063, -1.6675775, -1.7545762, -1.9188975,
       -3.6257977, -3.7870157, -3.7500079, -3.8942378, -2.2271624],
      dtype=float32)]


In [18]:
# inspect internal graph weights
def inspect_weights(checkpoint_model_dir):
    reader = tf.train.NewCheckpointReader(checkpoint_model_dir)
    variables = reader.get_variable_to_shape_map()
    for var, shape in variables.items():
        trained_weight = reader.get_tensor(var)
        print("VarName: {}".format(var))
        print("VarShape: {}".format(shape))
        print("VarValue: {}".format(trained_weight))
inspect_weights("./FMRec/model.ckpt")

VarName: optimizergraph/interactiongraph/LinearComponent/bias__0/Adam_1
VarShape: [1]
VarValue: [0.]
VarName: optimizergraph/interactiongraph/LinearComponent/bias__0/Adam
VarShape: [1]
VarValue: [0.]
VarName: optimizergraph/interactiongraph/LinearComponent/FC__0/Adam
VarShape: [4, 1]
VarValue: [[ 0.23304524]
 [-0.12927172]
 [ 0.6974491 ]
 [ 0.27985466]]
VarName: optimizergraph/interactiongraph/InteractiveComponent/interaction_factors/Adam
VarShape: [4, 5]
VarValue: [[ 0.649344   -0.6558524  -0.62966037 -0.6665276   0.65049756]
 [ 0.25947928 -0.26228905 -0.25123242 -0.26651993  0.26001492]
 [-0.52085096  0.4986877   0.5165919   0.5151211  -0.5005057 ]
 [-0.1103515   0.10308744  0.10525694  0.10739166 -0.10240713]]
VarName: optimizergraph/beta2_power
VarShape: []
VarValue: 0.903888463973999
VarName: optimizergraph/beta1_power
VarShape: []
VarValue: 2.3905195121187717e-05
VarName: interactiongraph/LinearComponent/FC__0
VarShape: [4, 1]
VarValue: [[-0.07031129]
 [-0.57071304]
 [ 0.22640811